# Function for MultiProcessing NCEP Data

## Request file from NCEP and pull the desired variables

In [ ]:
import os
import requests
import xarray as xr
from multiprocessing import Pool
import multiprocessing as mp
from IPython.display import clear_output


def process_file(file,VR,P,G,PR,T,i,meta,U10,U20,U30,U40,U50,U80,U100,V10,V20,V30,V40,V50,V80,V100,pix,latitude,longitude):
    # Downloading each file
    filename = os.path.basename(file)
    save_dir = '/Users/trak/Downloads/TEMPOOL/'
    file_path = save_dir + filename
    if not os.path.exists(file_path):
        print('Downloading', file)
        req = requests.get(file, allow_redirects=True)
        with open(filename, 'wb') as f:
            f.write(req.content)
    
    # Selecting File
    # Assuming `save_dir` is predefined and accessible
    save_dir = '/Users/trak/Downloads/TEMPOOL/'
    file_path = save_dir + filename
    
    # Open the data: VRATE,PBLH,GUST,TEMP,PRATE
    ds = xr.open_dataset(file_path, backend_kwargs={'filter_by_keys':{'typeOfLevel': ['planetaryBoundaryLayer','surface']}})
    vrate = ds["VRATE"].sel(latitude=latitude, longitude=longitude, method='nearest')
    pblh  = ds["unknown"].sel(latitude=latitude, longitude=longitude, method='nearest')
    gust  = ds["gust"].sel(latitude=latitude, longitude=longitude, method='nearest')
    temp= ds['t'].sel(latitude=latitude, longitude=longitude, method='nearest')
    prate= ds['prate'].sel(latitude=latitude, longitude=longitude, method='nearest')
    
    #Open the data: u,v for each level
    grib_data = xr.open_dataset(file_path, engine="cfgrib",filter_by_keys={'typeOfLevel': 'heightAboveGround','level':10})
    u10= grib_data["u10"].sel(latitude=latitude, longitude=longitude, method='nearest')
    v10= grib_data["v10"].sel(latitude=latitude, longitude=longitude, method='nearest')

    
    grib_data = xr.open_dataset(file_path, engine="cfgrib",filter_by_keys={'typeOfLevel': 'heightAboveGround','level':20})
    u20= grib_data["u"].sel(latitude=latitude, longitude=longitude, method='nearest')
    v20= grib_data["v"].sel(latitude=latitude, longitude=longitude, method='nearest')

    
    grib_data = xr.open_dataset(file_path, engine="cfgrib",filter_by_keys={'typeOfLevel': 'heightAboveGround','level':30})
    u30= grib_data["u"].sel(latitude=latitude, longitude=longitude, method='nearest')
    v30= grib_data["v"].sel(latitude=latitude, longitude=longitude, method='nearest')

    
    grib_data = xr.open_dataset(file_path, engine="cfgrib",filter_by_keys={'typeOfLevel': 'heightAboveGround','level':40})
    u40= grib_data["u"].sel(latitude=latitude, longitude=longitude, method='nearest')
    v40= grib_data["v"].sel(latitude=latitude, longitude=longitude, method='nearest')

    
    grib_data = xr.open_dataset(file_path, engine="cfgrib",filter_by_keys={'typeOfLevel': 'heightAboveGround','level':50})
    u50= grib_data["u"].sel(latitude=latitude, longitude=longitude, method='nearest')
    v50= grib_data["v"].sel(latitude=latitude, longitude=longitude, method='nearest')

    
    grib_data = xr.open_dataset(file_path, engine="cfgrib",filter_by_keys={'typeOfLevel': 'heightAboveGround','level':80})
    u80= grib_data["u"].sel(latitude=latitude, longitude=longitude, method='nearest')
    v80= grib_data["v"].sel(latitude=latitude, longitude=longitude, method='nearest')


    grib_data = xr.open_dataset(file_path, engine="cfgrib",filter_by_keys={'typeOfLevel': 'heightAboveGround','level':100})
    u100= grib_data["u100"].sel(latitude=latitude, longitude=longitude, method='nearest')
    v100= grib_data["v100"].sel(latitude=latitude, longitude=longitude, method='nearest')


    
    
    # Print the vrate values
    print("vrate: ",vrate.values)
    print("hpbl: ",pblh.values)
    print("wind speed: ",gust.values)
    print('lat',latitude)
    print('lon',longitude)
    print('id',pix)
    # Add each value to the list
    VR[i] = str(vrate.values)
    P[i] = str(pblh.values)
    G[i] = str(gust.values)
    PR[i] = str(prate.values)
    T[i] = str(temp.values)
    U10[i] = str(u10.values)
    V10[i] = str(v10.values)
    U20[i] = str(u20.values)
    V20[i] = str(v20.values)
    U30[i] = str(u30.values)
    V30[i] = str(v30.values)
    U40[i] = str(u40.values)
    V40[i] = str(v40.values)
    U50[i] = str(u50.values)
    V50[i] = str(v50.values)
    U80[i] = str(u80.values)
    V80[i] = str(v80.values)
    U100[i] = str(u100.values)
    V100[i] = str(v100.values)
    meta[i] = str(filename)
    clear_output(wait=True)
    

## Semaphore for handling Multiprocessing

In [ ]:
from multiprocessing import Process,Manager,Semaphore


def wrapper_process_file(*args):
    semaphore = mp.Semaphore(20)
    with semaphore:
        process_file(*args)